In [1]:
import requests, re
from bs4 import BeautifulSoup

In [2]:
def url_gen(ref,servings=1, metric = 'true' ):
    tail= 'print/?recipeType=Recipe&servings={}&isMetric={}'.format(servings,metric)
    url = 'https://www.allrecipes.com/recipe/' + ref+tail
    return url

In [3]:
def get_ingredients(ref): 
    
    url = url_gen(ref)
    r=requests.get(url)
    
    ingredients = []
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml')    
    col1 = soup.find('h2').next_sibling.next_sibling
    col2 = col1.next_sibling.next_sibling

    for i in col1.findAll('li'):
        last=str(i.contents).rfind('r')
        first=str(i.contents).find('n')
        ingredients.append((str(i.contents)[(first+1):(last-1)].lstrip()))
    for i in col2.findAll('li'):
        last=str(i.contents).rfind('r')
        first=str(i.contents).find('n')
        ingredients.append((str(i.contents)[(first+1):(last-1)].lstrip()))
    return ingredients

In [56]:
def get_nutrition(ref):
#Returns a dictionnary with nutritional values

    url = 'https://www.allrecipes.com/recipe/' + ref + 'fullrecipenutrition/'
    r=requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml') 
    info = soup.findAll('div', class_='nutrition-row')
    title = soup.find('h2').contents[0]
    nutrition = {}
    for i in info:
        a=i.find(class_='nutrient-name')
        b = re.findall('>(.*?):', str(a))[0]
        c = re.findall('value">(.*?)<', str(a))[0]
       # print(str(b) + ' : ' + str(c))
        nutrition[b] = c
    return title, nutrition

In [57]:
class Recipe:
    def __init__(self, name):
        self.name = name
        self.ingredients = get_ingredients(name)
        self.title, self.nutrition = get_nutrition(name)
    def display(self):
        print('\n ----------------- \n')
        print(self.title)
        print('\n')
        print(self.ingredients)
        print('\n')
        print(self.nutrition)

In [58]:
ref = '246179/black-chana-with-potato/'
ref = '45736/chicken-tikka-masala/'
#get_ingredients(ref)
a = Recipe(ref)
a.display()


 ----------------- 

Chicken Tikka Masala


['60 ml yogurt', '4 ml lemon juice', '1 g fresh ground cumin', '0.6 g ground cinnamon', '0.9 g cayenne pepper', '1 g freshly ground black pepper', '1 g minced fresh ginger', '2 g salt, or to taste', '3/4 boneless skinless chicken breasts, cut into bite-size pieces', '1 long skewers', '4 g butter', '1/4 clove garlic, minced', '1/4 jalapeno pepper, finely chopped', '1 g ground cumin', '1 g paprika', '2 g salt, or to taste', '1/4 (8 ounce) can tomato sauce', '60 ml heavy cream', '3 g chopped fresh cilantro']


{'Total Fat': '28.9g', 'Saturated Fat': '17.0g', 'Cholesterol': '143mg', 'Sodium': '1592mg', 'Potassium': '660mg', 'Total Carbohydrates': '13.3g', 'Dietary Fiber': '2.5g', 'Protein': '24.6g', 'Sugars': '7g', 'Vitamin A': '2432IU', 'Vitamin C': '11mg', 'Calcium': '206mg', 'Iron': '3mg', 'Thiamin': '0mg', 'Niacin': '13mg', 'Vitamin B6': '1mg', 'Magnesium': '57mg', 'Folate': '24mcg'}


In [36]:
def get_recipes(themeUrl, number = 20):
    
    r=requests.get(themeUrl) #add page 1,2,etc
    
    baseUrl = 'https://www.allrecipes.com/recipe/'
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml') 
    results = soup.findAll('h3', class_='fixed-recipe-card__h3')
    
    theme = []
    iteration = 0
    for i in results :
        
        if iteration  > number:
            break
            
        iteration = iteration + 1
        
        link = str(i.find('a').get('href'))
        first=link.find(baseUrl)
        theme.append(link[(first+len(baseUrl)):])
        
    return theme

In [61]:
themeUrl = 'https://www.allrecipes.com/recipes/233/world-cuisine/asian/indian/' 
themeUrl = 'https://www.allrecipes.com/recipes/728/world-cuisine/latin-american/mexican/'
themeLinks = get_recipes(themeUrl, number = 5)
themeRecipes = []

for recipe in themeLinks:
    themeRecipes.append(Recipe(recipe))

In [62]:
for i in themeRecipes:
    i.display()


 ----------------- 

Mexican Rice II


['10 ml vegetable oil', '45 g uncooked long-grain rice', '1 g garlic salt', '0.3 g ground cumin', '10 g chopped onion', '30 ml tomato sauce', '120 ml chicken broth']


{'Total Fat': '11g', 'Saturated Fat': '2.0g', 'Cholesterol': '2mg', 'Sodium': '1096mg', 'Potassium': '173mg', 'Total Carbohydrates': '42.4g', 'Dietary Fiber': '1.2g', 'Protein': '4.8g', 'Sugars': '2g', 'Vitamin A': '110IU', 'Vitamin C': '3mg', 'Calcium': '23mg', 'Iron': '2mg', 'Thiamin': '0mg', 'Niacin': '3mg', 'Vitamin B6': '0mg', 'Magnesium': '24mg', 'Folate': '86mcg'}

 ----------------- 

Spicy Chicken and Hominy Mexican Soup


['4 ml olive oil', '1/2 chicken breasts, cut into 1-inch pieces', '1/4 small onion, chopped', '1/2 chipotle peppers in adobo sauce, seeded and diced', '1/2 clove garlic, minced', '1/4 pinch garlic salt, or to taste', '1/4 (32 ounce) can enchilada sauce', '1/2 (16 ounce) can hominy', '1/4 (15 ounce) can diced tomatoes', '1/4 (15 ounce) can black beans, ri